# Final Project for Applied Physics in Programming
By: Eveliina Hampus, DIN22SP

1. Import necessary libraries. 

In [ ]:
import numpy as np # For calculations
import matplotlib.pyplot as plt # For plotting
import sympy as sm # For symbolic calculations
import scipy as sc # For scientific processing
from scipy.signal import butter,filtfilt
from math import radians, cos, sin, asin, sqrt
from sympy.abc import x